In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import zipfile

cv2.__version__

In [ ]:
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__

In [ ]:
#conect com google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#acessar e descompactar o zip do drive
path = "/content/gdrive/My Drive/Material.zip"
zip_object = zipfile.ZipFile(file = path, mode = "r")#r = read
zip_object.extractall('./')
zip_object.close

In [ ]:
base_imgs = 'Material/fer2013.zip'
zip_object = zipfile.ZipFile(file = base_imgs, mode = "r")#r = read
zip_object.extractall('./')
zip_object.close

In [ ]:
#acessando a base com fotos de expressoes faciais
data = pd.read_csv('fer2013/fer2013.csv')
#mostra os ultimos resultados do csv
data.tail()

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(data['emotion'], bins = 30)
plt.title('Imagnes x emoções')
#Classes: ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
pixels = data['pixels'].tolist()
pixels

In [ ]:
largura, altura = 48, 48
faces = []
amostras = 0
#formato de lista para numpy array
for pixel_sequence in pixels:
  face = [int(pixel) for pixel in pixel_sequence.split(' ')]
  face = np.asarray(face).reshape(largura, altura)
  faces.append(face)

  if (amostras < 10):
    cv2_imshow(face)
  amostras += 1

In [ ]:
print('Numero total de imagens do dataset: ', str(len(faces)))

In [ ]:
faces = np.asarray(faces)

In [ ]:
#qtd de imagens largura e altura
faces.shape

In [ ]:
#imagens com esaca de cinza adicionada no final
#quando coloca o -1 quer dizer que vai adiconar uma nova dimensão no final do vetor da matriz
faces = np.expand_dims(faces, -1)

In [ ]:
faces.shape

In [ ]:
faces = normalizar(faces)

In [ ]:
faces[0]

In [ ]:
#verificar emoção
emocoes = pd.get_dummies(data['emotion']).values

In [ ]:
#disgust neste caso
#essa rede neural tera 7 neuronios na camada de saida que ele ira indicar uma probabilidade de ele pertencer a cada uma dessas faces
emocoes[0]

In [ ]:
#imports tensorflow
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json

In [ ]:
#dividir em conjuntos para treinameto e validação
X_train, X_test, y_train, y_test = train_test_split(faces, emocoes, test_size = 0.1, random_state = 42)
_, X_val, _, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 41)

In [ ]:
print('Numero de imagnes no conjunto de treinamento: ', len(X_train))
print('Numero de imagnes no conjunto de teste: ', len(X_test))
print('Numero de imagnes no conjunto de validação: ', len(X_val))